# MQ-4

## Ro calculation

### Specifications

#### Standard Work Condition

    Vc (Circuit Voltage)       :  5v +/- 0.1
    Vh (Heating Voltage)       :  5v +/- 0.1
    Rl (Load Resistance)       :  20kOhm
    Rh (Heater Resistance)     :  33Ohm +/- 5%
    Ph (Heating consumption)   :  less than 750mw
    

#### Sensitivity Characteristic:

    Rs (Sensing Resistance)    : 1KOhm  -  60KOhm (1000ppm CH4)
    Preheat Time               : Over 24 hour
    
    Detecting Concentration Scope:
        CH4 and Natural Gas :  200ppm  - 10000ppm

In [1]:
#%pip install numpy

import numpy as np
import math

In [2]:
## Gas concentration values referenced in the datasheet

# Liquefied Petroleum Gas
lpg = {
    'p1': {'x': 200,   'y': 2.560},
    'p2': {'x': 1000,  'y': 1.507},
    'p3': {'x': 5000,  'y': 0.900},
    'p4': {'x': 10000, 'y': 0.743}
}

# Methane
ch4 = {
    'p1': {'x': 200,   'y': 1.753},
    'p2': {'x': 1000,  'y': 1.000},
    'p3': {'x': 5000,  'y': 0.567},
    'p4': {'x': 10000,  'y': 0.438}
}

# Hydrogen
h2 = {
    'p1': {'x': 200,   'y': 3.761},
    'p2': {'x': 1000,  'y': 2.832},
    'p3': {'x': 5000,  'y': 2.159},
    'p4': {'x': 10000, 'y': 1.891}
}

# Carbon Monoxide
co = {
    'p1': {'x': 200,   'y': 4.267},
    'p2': {'x': 1000,  'y': 3.785},
    'p3': {'x': 5000,  'y': 3.622},
    'p4': {'x': 10000, 'y': 3.509}
}

alcohol = {
    'p1': {'x': 200,   'y': 4.032},
    'p2': {'x': 1000,  'y': 3.622},
    'p3': {'x': 5000,  'y': 3.192},
    'p4': {'x': 10000, 'y': 3.054}
}

smoke = {
    'p1': {'x': 200,   'y': 3.906},
    'p2': {'x': 1000,  'y': 3.531},
    'p3': {'x': 5000,  'y': 2.886},
    'p4': {'x': 10000, 'y': 2.528}
}

AIR = 4.432

In [3]:
# RS value calculation
Rs    = lambda Vcc, Vrl, RL : ((Vcc * RL) / Vrl) - RL

# R0 value calculation
R0    = lambda Rs, air : Rs / air

# RS/Ro value calculation (ratio)
Ratio = lambda Rs, R0: Rs / R0

# log(y) = m.log(x) + b
# m
M     = lambda p1, p2 : (math.log10( p2['y'] / p1['y'] )) / (math.log10(p2['x']/p1['x']))
# b
B     = lambda p, m : math.log10(p['y']) - (m * math.log10(p['x']))

# PPM value calculation 
PPM   = lambda  ratio, m, b: 10 ** ((math.log10(ratio) - b) / m)

In [4]:
## Calculate the points to be used for the calculation of _m_ and _b_ according to the _ratio_ value 
def getPoints(racio, values):
    size = len(values)
    for idx in range(size - 1):
        if racio >= values[idx]['y']:
            if idx == 0:
                return (values[0], values[1])
            return(values[idx - 1], values[idx])
    return(values[size - 2], values[size - 1])

In [5]:
points = ['p1', 'p2', 'p3', 'p4']

_lpg     = [ lpg[n]     for n in list(lpg)     if n in points]
_ch4     = [ ch4[n]     for n in list(ch4)     if n in points]
_h2      = [ h2[n]      for n in list(h2)      if n in points]
_co      = [ co[n]      for n in list(co)      if n in points]
_alcohol = [ alcohol[n] for n in list(alcohol) if n in points]
_smoke   = [ smoke[n]   for n in list(smoke)   if n in points]

In [6]:
_ratios     = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
lpg_ppm     = []
ch4_ppm     = []
h2_ppm      = []
co_ppm      = []
alcohol_ppm = []
smoke_ppm   = []

for r in _ratios:
    
    p1, p2  = getPoints(r, _lpg)
    m = M(p1,p2)
    b = B(p1, m)
    lpg_ppm.append(PPM(r, m, b))
        
    p1, p2  = getPoints(r, _ch4)
    m = M(p1,p2)
    b = B(p1, m)
    ch4_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _h2)
    m = M(p1,p2)
    b = B(p1, m)
    h2_ppm.append(PPM(r, m, b))

    p1, p2  = getPoints(r, _co)
    m = M(p1,p2)
    b = B(p1, m)
    co_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _alcohol)
    m = M(p1,p2)
    b = B(p1, m)
    alcohol_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _smoke)
    m = M(p1,p2)
    b = B(p1, m)
    smoke_ppm.append(PPM(r, m, b))

    
print ("{:<8} {:<13} {:<13} {:<13} {:<15} {:<15} {:<15}"
       .format('Ratio','LPG', 'CH4', 'H2', 'CO', 'Alcohol', 'Smoke'))
for i in range(len(_ratios)):
    if _ratios[i] > 1:
        print ("{:<8} {:<13} {:<13} {:<13} {:<15} {:<15} {:<15}"
               .format( _ratios[i],
                       round(lpg_ppm[i],3),
                       round(ch4_ppm[i],3),
                       round(h2_ppm[i],3),
                       round(co_ppm[i],3),
                       round(alcohol_ppm[i],3),
                       round(smoke_ppm[i],3),
                      ))
    elif  _ratios[i] > 0.4:
        print ("{:<8} {:<13} {:<13} {:<13} {:<15} {:<15} {:<15}"
       .format( _ratios[i],
               round(lpg_ppm[i],3),
               round(ch4_ppm[i],3),
               round(h2_ppm[i],3),
               "----",
               "----",
               round(smoke_ppm[i],3),
              ))
    else :
        print ("{:<8} {:<13} {:<13} {:<13} {:<15} {:<15} {:<15}"
               .format( _ratios[i],
                       round(lpg_ppm[i],3),
                       round(ch4_ppm[i],3),
                       "----",
                       "----",
                       "----",
                       "----",
                      ))


Ratio    LPG           CH4           H2            CO              Alcohol         Smoke          
10       3.189         1.358         0.779         0.002           0.0             0.0            
9        4.392         1.837         1.417         0.009           0.001           0.0            
8        6.281         2.574         2.764         0.043           0.007           0.002          
7        9.422         3.775         5.895         0.26            0.051           0.018          
6        15.048        5.873         14.134        2.058           0.513           0.213          
5        26.181        9.906         39.762        23.802          7.916           3.9            
4        51.563        18.784        141.007       476.244         225.407         136.882        
3        123.542       42.855        721.135       307934.13       13228.591       3670.491       
2        423.313       137.053       7459.608      2184826167.78   7643119.762     34079.919      
1        3

### Empty $V_{RL}$ Values

The minimum $V_{RL}$ voltage values within the obtained values were considered. 
The measurement took place for +/- 4 days in an empty room with the door closed (no one entered the room) and the windows open to ensure air circulation.

- __Sensor 1__: 1.05 Volts
- __Sensor 2__: 1.08 Volts

In [7]:
# MQ 4 - sensor 1
sensor1 = {}
sensor1['Vcc'] = 5      #volts
sensor1['Vrl'] = 1.09  #volts
sensor1['RL']  = 10     #kOhms
sensor1['AIR'] = AIR  

sensor1['rs'] = Rs(sensor1['Vcc'], sensor1['Vrl'], sensor1['RL'])
sensor1['ro'] = R0(sensor1['rs'], sensor1['AIR'])
sensor1['ratio'] = Ratio(sensor1['rs'], sensor1['ro'])

# MQ 4 - sensor 2
sensor2 = {}

sensor2['Vcc'] = 5      #volts
sensor2['Vrl'] = 1.13   #volts
sensor2['RL']  = 10     #kOhms
sensor2['AIR'] = AIR   

sensor2['rs'] = Rs(sensor2['Vcc'], sensor2['Vrl'], sensor2['RL'])
sensor2['ro'] = R0(sensor2['rs'], sensor2['AIR'])
sensor2['ratio'] = Ratio(sensor2['rs'], sensor2['ro'])



print ("{:<8} {:<8} {:<8} {:<8}".format('Sensor','Rs', 'Ro','Ratio'))
print ("{:<8} {:<8} {:<8} {:<8}".format(1, round(sensor1['rs'],2), round(sensor1['ro'], 2), round(sensor1['ratio'], 2)))
print ("{:<8} {:<8} {:<8} {:<8}".format(2, round(sensor2['rs'],2), round(sensor2['ro'], 2), round(sensor2['ratio'], 2)))

Sensor   Rs       Ro       Ratio   
1        35.87    8.09     4.43    
2        34.25    7.73     4.43    


In [8]:
## PPM
# Concentration of gases in ppm for the values of Rs/Ro (ratio) in Clean Air

sensor1['lpg'] = {}
sensor1['lpg']['p1'], sensor1['lpg']['p2'] = getPoints(sensor1['ratio'], _lpg)
sensor1['lpg']['m'] = M(sensor1['lpg']['p1'], sensor1['lpg']['p2'])
sensor1['lpg']['b'] = B(sensor1['lpg']['p1'], sensor1['lpg']['m'])

sensor1['ch4'] = {}
sensor1['ch4']['p1'], sensor1['ch4']['p2'] = getPoints(sensor1['ratio'], _ch4)
sensor1['ch4']['m'] = M(sensor1['ch4']['p1'], sensor1['ch4']['p2'])
sensor1['ch4']['b'] = B(sensor1['ch4']['p1'], sensor1['ch4']['m'])

sensor1['h2'] = {}
sensor1['h2']['p1'], sensor1['h2']['p2'] = getPoints(sensor1['ratio'], _h2)
sensor1['h2']['m'] = M(sensor1['h2']['p1'], sensor1['h2']['p2'])
sensor1['h2']['b'] = B(sensor1['h2']['p1'], sensor1['h2']['m'])

sensor1['co'] = {}
sensor1['co']['p1'], sensor1['co']['p2'] = getPoints(sensor1['ratio'], _co)
sensor1['co']['m'] = M(sensor1['co']['p1'], sensor1['co']['p2'])
sensor1['co']['b'] = B(sensor1['co']['p1'], sensor1['co']['m'])


sensor1['alcohol'] = {}
sensor1['alcohol']['p1'], sensor1['alcohol']['p2'] = getPoints(sensor1['ratio'], _alcohol)
sensor1['alcohol']['m'] = M(sensor1['alcohol']['p1'], sensor1['alcohol']['p2'])
sensor1['alcohol']['b'] = B(sensor1['alcohol']['p1'], sensor1['alcohol']['m'])


sensor1['smoke'] = {}
sensor1['smoke']['p1'], sensor1['smoke']['p2'] = getPoints(sensor1['ratio'], _smoke)
sensor1['smoke']['m'] = M(sensor1['smoke']['p1'], sensor1['smoke']['p2'])
sensor1['smoke']['b'] = B(sensor1['smoke']['p1'], sensor1['smoke']['m'])

####

sensor2['lpg'] = {}
sensor2['lpg']['p1'], sensor2['lpg']['p2'] = getPoints(sensor2['ratio'], _lpg)
sensor2['lpg']['m'] = M(sensor2['lpg']['p1'], sensor2['lpg']['p2'])
sensor2['lpg']['b'] = B(sensor2['lpg']['p1'], sensor2['lpg']['m'])

sensor2['ch4'] = {}
sensor2['ch4']['p1'], sensor2['ch4']['p2'] = getPoints(sensor2['ratio'], _ch4)
sensor2['ch4']['m'] = M(sensor2['ch4']['p1'], sensor2['ch4']['p2'])
sensor2['ch4']['b'] = B(sensor2['ch4']['p1'], sensor2['ch4']['m'])

sensor2['h2'] = {}
sensor2['h2']['p1'], sensor2['h2']['p2'] = getPoints(sensor2['ratio'], _h2)
sensor2['h2']['m'] = M(sensor2['h2']['p1'], sensor2['h2']['p2'])
sensor2['h2']['b'] = B(sensor2['h2']['p1'], sensor2['h2']['m'])

sensor2['co'] = {}
sensor2['co']['p1'], sensor2['co']['p2'] = getPoints(sensor2['ratio'], _co)
sensor2['co']['m'] = M(sensor2['co']['p1'], sensor2['co']['p2'])
sensor2['co']['b'] = B(sensor2['co']['p1'], sensor2['co']['m'])


sensor2['alcohol'] = {}
sensor2['alcohol']['p1'], sensor2['alcohol']['p2'] = getPoints(sensor2['ratio'], _alcohol)
sensor2['alcohol']['m'] = M(sensor2['alcohol']['p1'], sensor2['alcohol']['p2'])
sensor2['alcohol']['b'] = B(sensor2['alcohol']['p1'], sensor2['alcohol']['m'])


sensor2['smoke'] = {}
sensor2['smoke']['p1'], sensor2['smoke']['p2'] = getPoints(sensor2['ratio'], _smoke)
sensor2['smoke']['m'] = M(sensor2['smoke']['p1'], sensor2['smoke']['p2'])
sensor2['smoke']['b'] = B(sensor2['smoke']['p1'], sensor2['smoke']['m'])


####

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Volt', 'Ratio','LPG', 'CH4', 'H2', 'CO', 'Alcohol', 'Smoke'))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor1['Vrl'], 3),
           round(sensor1['ratio'], 2),
           round(PPM(sensor1['ratio'], sensor1['lpg']['m'],     sensor1['lpg']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['ch4']['m'],     sensor1['ch4']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['h2']['m'],      sensor1['h2']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['co']['m'],      sensor1['co']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['alcohol']['m'], sensor1['alcohol']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['smoke']['m'],   sensor1['smoke']['b']),2)
          ))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor2['Vrl'], 3),
           round(sensor2['ratio'], 2),
           round(PPM(sensor2['ratio'], sensor2['lpg']['m'],     sensor2['lpg']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['ch4']['m'],     sensor2['ch4']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['h2']['m'],      sensor2['h2']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['co']['m'],      sensor2['co']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['alcohol']['m'], sensor2['alcohol']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['smoke']['m'],   sensor2['smoke']['b']),2),
          ))    

Volt     Ratio    LPG           CH4           H2            CO            Alcohol       Smoke        
1.09     4.43     37.76         14.0          78.81         120.17        48.36         26.68        
1.13     4.43     37.76         14.0          78.81         120.17        48.36         26.68        


In [9]:
## Projection - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
vrl    = np.arange(sensor1['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor1['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('','Volt', 'Ratio','LPG', 'CH4', 'H2', 'CO', 'Alcohol', 'Smoke'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])

    projection['h2'] = {}
    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])

    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])


    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])


    projection['smoke'] = {}
    projection['smoke']['p1'], projection['smoke']['p2'] = getPoints(r, _smoke)
    projection['smoke']['m'] = M(projection['smoke']['p1'], projection['smoke']['p2'])
    projection['smoke']['b'] = B(projection['smoke']['p1'], projection['smoke']['m'])


    if r > 1.95:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                   round(PPM(r, projection['smoke']['m'], projection['smoke']['b']),2)
                  ))
    elif r > 1.25:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
            .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                   round(PPM(r, projection['smoke']['m'], projection['smoke']['b']),2)
                  ))
    elif r > 0.18:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
            .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   "----",
                   "----",
                   round(PPM(r, projection['smoke']['m'], projection['smoke']['b']),2)
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
            .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                   "----",
                   "----",
                   round(PPM(r, projection['smoke']['m'], projection['smoke']['b']),2)
                  ))


    Volt     Ratio    LPG           CH4           H2            CO            Alcohol       Smoke        
0   1.09     4.43     37.76         14.0          78.81         120.17        48.36         26.68        
1   1.19     3.96     53.34         19.39         150.19        552.99        266.38        163.46       
2   1.29     3.55     73.88         26.38         276.05        7599.87       1276.11       904.43       
3   1.39     3.21     100.71        35.34         492.32        70701.47      4676.78       2145.72      
4   1.49     2.91     135.44        46.74         856.29        597113.12     21270.03      4673.64      
5   1.59     2.65     180.12        61.18         1483.68       4650719.13    92702.38      7817.72      
6   1.69     2.42     237.3         79.36         2541.76       33846376.86   384837.93     12570.88     
7   1.79     2.22     310.16        102.18        4287.93       232736671.93  1533890.63    19941.36     
8   1.89     2.03     402.76        130.76    

In [10]:
## Projection - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
vrl    = np.arange(sensor2['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor2['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('','Volt', 'Ratio','LPG', 'CH4', 'H2', 'CO', 'Alcohol', 'Smoke'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    
    projection['lpg'] = {}
    projection['lpg']['p1'], projection['lpg']['p2'] = getPoints(r, _lpg)
    projection['lpg']['m'] = M(projection['lpg']['p1'], projection['lpg']['p2'])
    projection['lpg']['b'] = B(projection['lpg']['p1'], projection['lpg']['m'])


    projection['ch4'] = {}
    projection['ch4']['p1'], projection['ch4']['p2'] = getPoints(r, _ch4)
    projection['ch4']['m'] = M(projection['ch4']['p1'], projection['ch4']['p2'])
    projection['ch4']['b'] = B(projection['ch4']['p1'], projection['ch4']['m'])

    projection['h2'] = {}
    projection['h2']['p1'], projection['h2']['p2'] = getPoints(r, _h2)
    projection['h2']['m'] = M(projection['h2']['p1'], projection['h2']['p2'])
    projection['h2']['b'] = B(projection['h2']['p1'], projection['h2']['m'])

    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])


    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])


    projection['smoke'] = {}
    projection['smoke']['p1'], projection['smoke']['p2'] = getPoints(r, _smoke)
    projection['smoke']['m'] = M(projection['smoke']['p1'], projection['smoke']['p2'])
    projection['smoke']['b'] = B(projection['smoke']['p1'], projection['smoke']['m'])


    if r > 1.95:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   round(PPM(r, projection['co']['m'], projection['co']['b']),2),
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                   round(PPM(r, projection['smoke']['m'], projection['smoke']['b']),2)
                  ))
    elif r > 1.25:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
            .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   "----",
                   round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
                   round(PPM(r, projection['smoke']['m'], projection['smoke']['b']),2)
                  ))
    elif r > 0.18:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
            .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   round(PPM(r, projection['h2']['m'], projection['h2']['b']),2),
                   "----",
                   "----",
                   round(PPM(r, projection['smoke']['m'], projection['smoke']['b']),2)
                  ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
            .format(idx,
                   round(vrl[idx], 3),
                   round(r, 2),
                   round(PPM(r, projection['lpg']['m'], projection['lpg']['b']),2),
                   round(PPM(r, projection['ch4']['m'], projection['ch4']['b']),2),
                   "----",
                   "----",
                   "----",
                   round(PPM(r, projection['smoke']['m'], projection['smoke']['b']),2)
                  ))


    Volt     Ratio    LPG           CH4           H2            CO            Alcohol       Smoke        
0   1.13     4.43     37.76         14.0          78.81         120.17        48.36         26.68        
1   1.23     3.97     52.9          19.24         147.91        533.24        255.77        156.55       
2   1.33     3.57     72.78         26.01         268.41        6820.75       1198.21       835.84       
3   1.43     3.23     98.65         34.65         473.66        60918.65      4288.27       2032.24      
4   1.53     2.93     132.04        45.63         816.58        497237.47     18653.45      4371.72      
5   1.63     2.68     174.91        59.5          1400.91       3763648.54    79648.19      7431.64      
6   1.73     2.45     229.66        76.95         2384.44       26742767.03   325018.53     11881.81     
7   1.83     2.24     299.35        98.82         4000.85       180261486.29  1277076.42    18758.59     
8   1.93     2.06     387.83        126.18    

In [11]:
## projectionRoRo - Fixing VRL and Varying the Ro

# Sensor 1

Vcc    = 5 #volts
Vrl    = sensor1['Vrl']
RL     = 10 #kOhms 

projectionRo = {}
projectionRo['rs'] = sensor1['rs']   
projectionRo['ro'] = np.arange(sensor1['ro'] - sensor1['ro']/5 , sensor1['ro'] + sensor1['ro']/5 , 0.1)
projectionRo['ratio'] = [ Ratio(projectionRo['rs'], ro) for ro in projectionRo['ro']] 

#print(projectionRoRo['rs'])
#print(projectionRoRo['ratio'])

print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
       .format('', 'Volt','Rs', 'Ro', 'Ratio','LPG', 'CH4', 'H2', 'CO', 'Alcohol', 'Smoke'))

for idx in range(len(projectionRo['ratio'])):

    r = projectionRo['ratio'][idx]
    
    projectionRo['lpg'] = {}
    projectionRo['lpg']['p1'], projectionRo['lpg']['p2'] = getPoints(r, _lpg)
    projectionRo['lpg']['m'] = M(projectionRo['lpg']['p1'], projectionRo['lpg']['p2'])
    projectionRo['lpg']['b'] = B(projectionRo['lpg']['p1'], projectionRo['lpg']['m'])


    projectionRo['ch4'] = {}
    projectionRo['ch4']['p1'], projectionRo['ch4']['p2'] = getPoints(r, _ch4)
    projectionRo['ch4']['m'] = M(projectionRo['ch4']['p1'], projectionRo['ch4']['p2'])
    projectionRo['ch4']['b'] = B(projectionRo['ch4']['p1'], projectionRo['ch4']['m'])

    projectionRo['h2'] = {}
    projectionRo['h2']['p1'], projectionRo['h2']['p2'] = getPoints(r, _h2)
    projectionRo['h2']['m'] = M(projectionRo['h2']['p1'], projectionRo['h2']['p2'])
    projectionRo['h2']['b'] = B(projectionRo['h2']['p1'], projectionRo['h2']['m'])

    projectionRo['co'] = {}
    projectionRo['co']['p1'], projectionRo['co']['p2'] = getPoints(r, _co)
    projectionRo['co']['m'] = M(projectionRo['co']['p1'], projectionRo['co']['p2'])
    projectionRo['co']['b'] = B(projectionRo['co']['p1'], projectionRo['co']['m'])


    projectionRo['alcohol'] = {}
    projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'] = getPoints(r, _alcohol)
    projectionRo['alcohol']['m'] = M(projectionRo['alcohol']['p1'], projectionRo['alcohol']['p2'])
    projectionRo['alcohol']['b'] = B(projectionRo['alcohol']['p1'], projectionRo['alcohol']['m'])


    projectionRo['smoke'] = {}
    projectionRo['smoke']['p1'], projectionRo['smoke']['p2'] = getPoints(r, _smoke)
    projectionRo['smoke']['m'] = M(projectionRo['smoke']['p1'], projectionRo['smoke']['p2'])
    projectionRo['smoke']['b'] = B(projectionRo['smoke']['p1'], projectionRo['smoke']['m'])


    print ("{:<3} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8} {:<8}"
       .format(idx,
               round(Vrl, 3),
               round(projectionRo['rs'],2),
               round(projectionRo['ro'][idx],2),
               round(r, 2),
               round(PPM(r, projectionRo['lpg']['m'], projectionRo['lpg']['b']),2),
               round(PPM(r, projectionRo['ch4']['m'], projectionRo['ch4']['b']),2),
               round(PPM(r, projectionRo['h2']['m'], projectionRo['h2']['b']),2),
               round(PPM(r, projectionRo['co']['m'], projectionRo['co']['b']),2),
               round(PPM(r, projectionRo['alcohol']['m'], projectionRo['alcohol']['b']),2),
               round(PPM(r, projectionRo['smoke']['m'], projectionRo['smoke']['b']),2)
              ))

    Volt     Rs       Ro       Ratio    LPG      CH4      H2       CO       Alcohol  Smoke   
0   1.09     35.87    6.48     5.54     19.17    7.38     22.22    6.01     1.7      0.76    
1   1.09     35.87    6.58     5.46     20.09    7.71     24.24    7.38     2.14     0.97    
2   1.09     35.87    6.68     5.37     21.03    8.06     26.41    9.04     2.68     1.23    
3   1.09     35.87    6.78     5.29     22.0     8.41     28.73    11.03    3.35     1.56    
4   1.09     35.87    6.88     5.22     23.0     8.77     31.22    13.43    4.18     1.98    
5   1.09     35.87    6.98     5.14     24.03    9.14     33.89    16.31    5.19     2.49    
6   1.09     35.87    7.08     5.07     25.1     9.52     36.74    19.74    6.42     3.12    
7   1.09     35.87    7.18     5.0      26.19    9.91     39.78    23.83    7.93     3.91    
8   1.09     35.87    7.28     4.93     27.31    10.31    43.03    28.7     9.76     4.87    
9   1.09     35.87    7.38     4.86     28.47    10.72    46

### Ro values to be considered

- __Sensor 1__: 8.49 kOhms
- __Sensor 2__: 8.19 kOhms